# Especialização em Ciência de Dados - PUC-Rio
# Machine Learning
# Projeto completo de Classificação Binária

## 1. Definição do Problema

O dataset usado neste projeto será o ***Sonar Mines vs Rocks***, disponível em https://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+(Sonar,+Mines+vs.+Rocks). 

O problema consiste em detectar se os objetos detectados pelo sonar são de metal ou de rocha. Cada exemplo contém 60 atributos numéricos no intervalo de 0.0 a 1.0. Cada número representa a energia dentro de uma determinada faixa de frequência, integrada durante um certo período de tempo. O rótulo associado a cada exemplo contém a letra R se o objeto é uma rocha e M se é um mina. Os números nos rótulos estão em ordem crescente de ângulo de aspecto, mas eles não codificam o ângulo diretamente.

## 2. Carga de Dados

In [0]:
# Imports
import numpy
from matplotlib import pyplot
from matplotlib import cm
from pandas import read_csv
from pandas import set_option
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

Faça download do dataset no site do repositório do UCI Machine Learning, salvando no diretório de trabalho local com o nome de arquivo sonar.all-data.csv.

Não iremos especificar os nomes dos atributos porque eles ​​não têm nomes significativos. Também indicamos que não há informações de cabeçalho (header), para evitar que o código de carregamento de arquivo tome o primeiro registro como o nome da coluna. 

Com o dataset carregado, iremos explorá-lo um pouco.

In [0]:
# Carga do dataset
url = 'sonar.all-data.csv'
dataset = read_csv(url, header=None)

## 3. Análise de Dados

### 3.1. Estatísticas Descritivas

In [0]:
# dimensões do dataset
print(dataset.shape)

In [0]:
# tipos de cada atributo
set_option('display.max_rows', 100) 
print(dataset.dtypes)

Agora, vamos dar uma olhada nas 20 primeiras linhas dos dados. Isso não mostra todas as colunas, mas podemos ver que todos os dados têm a mesma escala. Também podemos ver que o atributo de classe (60) possui valores de string.

In [0]:
# head
set_option('display.width', 100) 
print(dataset.head(20))

Analisando rapidamente os valores de classe, podemos ver que as classes são razoavelmente equilibradas entre M (minas) e R (rochas).

In [0]:
# distribuição das classes
print(dataset.groupby(60).size())

### 3.2. Visualizações Unimodais

Vamos agora gerar visualizações dos atributos individualmente. 

Primeiro, vamos gerar os histogramas de cada atributo para ter uma ideia das distribuições de dados. Veremos que existem algumas distribuições do tipo Gaussiana e algumas distribuições do tipo exponencial.

In [0]:
# histogramas
dataset.hist(sharex=False, sharey=False, xlabelsize=1, ylabelsize=1, figsize=(12,8))
pyplot.show()

Vamos dar uma olhada na mesma perspectiva dos dados usando gráficos de densidade. Veremos que muitos dos atributos têm uma distribuição distorcida. Uma transformação como a Box-Cox que pode aproximar a distribuição de uma Normal pode ser útil.

In [0]:
# density plots
dataset.plot(kind='density', subplots=True, layout=(8,8), sharex=False, legend=True, fontsize=1, figsize=(12,8))
pyplot.show()

### 3.3. Visualizações Multimodais

Ao visualizar as correlações entre os atributos, perceberemos que parece haver alguma estrutura na ordem dos atributos. O azul ao redor da diagonal sugere que os atributos que estão próximos um do outro são geralmente mais correlacionados entre si. Os vermelhos também sugerem alguma correlação negativa moderada, a medida que os atributos estão mais distantes um do outro na ordenação.

In [0]:
# Matriz de Correlação
fig = pyplot.figure(figsize=(12,8))
ax = fig.add_subplot(111)
cax = ax.matshow(dataset.corr(), vmin=-1, vmax=1, interpolation='none', cmap=cm.get_cmap('RdBu')) 
fig.colorbar(cax)
pyplot.show()

## 4. Pré-Processamento dos dados

### 4.1. Conjunto de validação

É uma boa prática usar um conjunto de validação (na literatura também chamado de conjunto de testes), uma amostra dos dados que não será usada para a modelagem, mas somente no fim do projeto para confirmar a precisão do  modelo final. É um teste  que podemos usar para verificar se erramos e para nos dar confiança nas estimativas em dados não vistos. Usaremos 80% do conjunto de dados para modelagem e guardaremos 20% para validação.

In [0]:
# Separação em conjuntos de treino e validação
array = dataset.values
X = array[:,0:60].astype(float)
Y = array[:,60]
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y,
    test_size=validation_size, random_state=seed)

## 5. Modelos de Classificação

### 5.1. Criação e avaliação de modelos: linha base

Não sabemos quais modelos performarão bem neste conjunto de dados. A intuição sugere que algoritmos baseados em distância, como KNN e SVM podem se sair bem. 

Para testar, usaremos a validação cruzada 10-fold e avaliaremos os modelos usando a métrica de acurácia.

In [0]:
# Parâmetros
num_folds = 10
seed = 7
scoring = 'accuracy'

Vamos criar uma linha base de desempenho para esse problema e verificar vários modelos diferentes com suas configurações padrão: Regressão Logística, Árvores de classificação e regressão (CART), Máquinas de vetores de suporte (SVM), Naive Bayes (NB) e k-vizinhos mais próximos (KNN).

In [0]:
# Criação dos modelos
models = []
models.append(('LR', LogisticRegression(solver='liblinear'))) 
models.append(('KNN', KNeighborsClassifier())) 
models.append(('CART', DecisionTreeClassifier())) 
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))

Agora vamos comparar os modelos, exibindo a acurácia média e o desvio padrão de cada um:

In [0]:
# Avaliação dos modelos
results = []
names = []
for name, model in models:
  kfold = KFold(n_splits=num_folds, random_state=seed)
  cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
  results.append(cv_results)
  names.append(name)
  msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
  print(msg)

Os resultados sugerem que tanto a regressão logística quanto os vizinhos mais próximos k podem ser bons modelos, mas estes são apenas valores médios de acurácia, sendo também prudente observar a distribuição dos resultados de cada fold da validação cruzada.

In [0]:
# Comparação dos modelos
fig = pyplot.figure() 
fig.suptitle('Comparação dos Modelos') 
ax = fig.add_subplot(111) 
pyplot.boxplot(results) 
ax.set_xticklabels(names) 
pyplot.show()

Os resultados mostram uma distribuição restrita para o KNN, o que é encorajador, sugerindo baixa variação. Os maus resultados para o SVM são surpreendentes.

É possível que a distribuição variada dos atributos esteja afetando a acurácia de algoritmos como o SVM. A seguir, repetiremos este processo usando uma cópia padronizada do conjunto de dados de treinamento.

### 5.2. Criação e avaliação de modelos: dados padronizados

Como suspeitamos que as diferentes distribuições dos dados brutos possam impactar negativamente a habilidade de alguns modelos, vamos agora utilizar cópia padronizada do dataset. Os dados serão transformados de modo que cada atributo tenha média 0 e um desvio padrão 1. 

Para evitar o "vazamento de dados" na transformação, vamos usar pipelines que padronizam os dados e constroem o modelo para cada fold de teste de validação cruzada. Dessa forma, podemos obter uma estimativa justa de como cada modelo com dados padronizados pode funcionar com dados não vistos.

In [0]:
# Padronização do dataset
pipelines = []
pipelines.append(('ScaledLR', Pipeline([('Scaler', StandardScaler()),('LR', LogisticRegression(solver='liblinear'))]))) 
pipelines.append(('ScaledKNN', Pipeline([('Scaler', StandardScaler()),('KNN', KNeighborsClassifier())])))
pipelines.append(('ScaledCART', Pipeline([('Scaler', StandardScaler()),('CART', DecisionTreeClassifier())])))
pipelines.append(('ScaledNB', Pipeline([('Scaler', StandardScaler()),('NB', GaussianNB())])))
pipelines.append(('ScaledSVM', Pipeline([('Scaler', StandardScaler()),('SVM', SVC(gamma='auto'))])))
results = []
names = []
for name, model in pipelines:
  kfold = KFold(n_splits=num_folds, random_state=seed)
  cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
  results.append(cv_results)
  names.append(name)
  msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
  print(msg)

Analisando os resultados, vemos que o KNN ainda está indo bem, ainda melhor do que antes. Também podemos ver que a padronização dos dados levou o SVM para o melhor modelo testado até agora.

Vamos analisar estes resultados graficamente:

In [0]:
# Comparação dos modelos com dados padronizados
fig = pyplot.figure()
fig.suptitle('Comparação dos modelos com dados padronizados') 
ax = fig.add_subplot(111) 
pyplot.boxplot(results) 
ax.set_xticklabels(names)
pyplot.show()

Os resultados sugerem que aprofundemos os modelos SVM e KNN, sendo muito provável que outras configurações possam render modelos ainda mais precisos.

### 5.3. Ajuste dos Modelos

#### Ajuste do KNN
Podemos começar ajustando o número de vizinhos e as métricas de distância para o KNN. Tentaremos todos os valores ímpares de k de 1 a 21 e as métricas de distância euclideana, manhattan e minkowski. Cada valor de k e de distância será avaliado usando a validação cruzada 10-fold no conjunto de dados padronizado.

In [0]:
# Tuning do KNN

scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)

k = [1,3,5,7,9,11,13,15,17,19,21]
distancias = ["euclidean", "manhattan", "minkowski"]
param_grid = dict(n_neighbors=k, metric=distancias)

model = KNeighborsClassifier()

kfold = KFold(n_splits=num_folds, random_state=seed)

grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold, iid=True)
grid_result = grid.fit(rescaledX, Y_train)
print("Melhor: %f usando %s" % (grid_result.best_score_, grid_result.best_params_)) 

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) com: %r" % (mean, stdev, param))

Vimos que a melhor configuração tem k = 1. Isso é interessante, pois o algoritmo fará previsões usando a instância mais semelhante.

#### Ajuste do SVM
Iremos ajustar dois parâmetros principais do algoritmo SVM, o valor de C (o quanto flexibilizar a margem) e o tipo de kernel. O padrão para o SVM (a classe SVC) é usar o kernel da Função Base Base Radial (RBF) com um valor C definido como 1.0. Cada combinação de valores será avaliada usando a validação cruzada 10-fold no conjunto de dados padronizado.

In [0]:
# Tuning do KNN

scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)

c_values = [0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.3, 1.5, 1.7, 2.0]
kernel_values = ['linear', 'poly', 'rbf', 'sigmoid']
param_grid = dict(C=c_values, kernel=kernel_values)

model = SVC(gamma='auto')

kfold = KFold(n_splits=num_folds, random_state=seed)

grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold, iid=True)
grid_result = grid.fit(rescaledX, Y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) 

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print("%f (%f) with: %r" % (mean, stdev, param))

Podemos ver que a configuração com maior acurácia foi o SVM com kernel RBF e C = 1,5. A melhor acurácia é aparentemente melhor do que o que o KNN poderia alcançar.

## 6. Métodos Ensemble

Outra maneira de melhorar o desempenho dos algoritmos é usar métodos de ensemble. Avaliaremos quatro modelos diferentes, sendo dois métodos de Boosting e dois de Bagging:
* Métodos de Boosting: AdaBoost (AB) e Gradient Boosting (GBM).
* Métodos de Bagging: Random Forests (RF) e Extra Trees (ET).

Usaremos novamente a validação cruzada 10-fold. Nenhuma padronização de dados será usada neste caso porque todos os quatro algoritmos de conjunto são baseados em árvores de decisão, que são modelos menos sensíveis às distribuições de dados.

In [0]:
# Ensembles

ensembles = []
ensembles.append(('AB', AdaBoostClassifier())) 
ensembles.append(('GBM', GradientBoostingClassifier())) 
ensembles.append(('RF', RandomForestClassifier(n_estimators=10))) 
ensembles.append(('ET', ExtraTreesClassifier(n_estimators=10))) 
results = []
names = []
for name, model in ensembles:
  kfold = KFold(n_splits=num_folds, random_state=seed)
  cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
  results.append(cv_results)
  names.append(name)
  msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
  print(msg)

Vamos comparar os resultados graficamente:

In [0]:
# Comparação de modelos
fig = pyplot.figure()
fig.suptitle('Comparação de modelos Ensemble') 
ax = fig.add_subplot(111) 
pyplot.boxplot(results) 
ax.set_xticklabels(names)
pyplot.show()

Os resultados sugerem que o GBM e o ET são provavelmente, os melhores modelos, podendo ser ainda aprimorados com variações de parâmetros.

## 7. Finalização do Modelo

Neste estudo, o SVM foi o modelo que mostrou melhor acurácia, melhor estabilidade, além de ser um modelo de baixa complexidade para este problema. 

Finalizaremos o modelo treinando-o em todo o conjunto de dados de treinamento e faremos predições para o conjunto de dados de validação (separado anteriormente)para confirmar nossas descobertas. 

Até o momento, vimos que o SVM tem um desempenho melhor quando o conjunto de dados é padronizado. Assim, iremos padronizar todo o conjunto de dados de treinamento e depois, aplicar a mesma transformação aos atributos de entrada do conjunto de dados de validação.

In [0]:
# Preparação do modelo
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
model = SVC(C=1.5)
model.fit(rescaledX, Y_train)

# Estimativa da acurácia no conjunto de validação
rescaledValidationX = scaler.transform(X_validation)
predictions = model.predict(rescaledValidationX)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

Podemos ver que alcançamos uma acurácia de quase 86% no conjunto de dados de validação, uma pontuação que se aproxima das nossas expectativas estimadas durante o ajuste do SVM.

## Resumo

Trabalhamos com um problema de aprendizado de máquina de modelagem preditiva de classificação de ponta a ponta. As etapas abordadas foram:

* Definição do problema (dados de retorno do sonar).
* Carga dos dados
* Análise dos dados (mesma escala, mas distribuições diferentes de dados).
* Avaliação de modelos (o KNN parecia bom).
* Avaliação de modelos com padronização (KNN e SVM pareciam bons).
* Ajuste dos modelos (K = 1 para KNN foi bom, SVM com um núcleo RBF e C = 1.5 foi melhor).
* Métodos de ensemble (bagging e boosting, não performaram tão bem quanto SVM).
* Finalização do modelo (use todos os dados de treinamento e valide usando o conjunto de dados de validação).